In [564]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import itertools

# HW2:  Модель Word2vec
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

## 0. Loading data

Составьте достаточно большую коллекцию текстов на любом языке или ис-пользуйте тексты отсюда:<br>
https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [565]:
text_dataset = pd.read_csv('data/unlabeledTrainData.csv', sep=';', encoding = 'latin1')

In [566]:
text_dataset.head(n = 5)

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor SpoilersIn New York, Joan Barnard (Elvir..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


In [567]:
def split_dataset_into_list_sentences(df, size = None):
    df_sentences_list = []
    if (size == None):
        size = len(df)
    for i in range(0, size):
        df_sentences_list.append(sent_tokenize(df.review.values[i]))
    return df_sentences_list

In [568]:
def flatten_list_of_lists(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

In [569]:
def split_sentence_into_words_list(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    return [x.lower() for x in tokenizer.tokenize(sentence)]

In [570]:
def split_list_of_sentences_into_words_list(list_sentences):
    list_sentences_words = []
    for sentence in list_sentences:
        list_sentences_words.append(split_sentence_into_words_list(sentence))
    return list_sentences_words

In [571]:
def process_dataset(dataset):
    list_sentences = split_dataset_into_list_sentences(dataset)
    list_sentences = flatten_list_of_lists(list_sentences)
    list_sentences_words = split_list_of_sentences_into_words_list(list_sentences)
    return list_sentences_words

In [572]:
%%time
processed_dataset = process_dataset(text_dataset)

CPU times: user 51.5 s, sys: 7.6 s, total: 59.1 s
Wall time: 1min 2s


## 1. Fitting Word2Vec model

Обучите модель word2vec. Оцените время обучения модели,используя модуль time. Есть два варианта обучения модели: по отзывам целиком и с уче-там границ предложений. В принципе, погрешностью, которая возникает в первом случае можно пренебречь, но если вы хотите учитывать границы предложений, то можно использовать sent_tokenize из nltk.tokenize

In [573]:
saved_model_name = 'savedmodel'

As Word2Vec tooks a lot of time to fit model, saving it to file

In [574]:
%%time
# model = Word2Vec(processed_dataset, workers=4)
# model.save(saved_model_name)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [575]:
model = Word2Vec.load(saved_model_name)

## 2. Looking for similar words

Примеры практически безошибочного (во всяком случае для ТОП-3) определения синонимов с помощью most_similar для следующих слов: 

**Amazing**

In [576]:
model.most_similar(positive = ['amazing'], topn = 3)

[('incredible', 0.8616176247596741),
 ('awesome', 0.8323751091957092),
 ('outstanding', 0.7665009498596191)]

**Stunning**

In [577]:
model.most_similar(positive = ['stunning'], topn = 3)

[('breathtaking', 0.8776392340660095),
 ('magnificent', 0.8123955130577087),
 ('dazzling', 0.8080258369445801)]

**Great**

In [578]:
model.most_similar(positive = ['great'], topn = 3)

[('wonderful', 0.8272826075553894),
 ('fantastic', 0.8226151466369629),
 ('terrific', 0.7967977523803711)]

**Thrilling**

In [550]:
model.most_similar(positive = ['thrilling'], topn = 3)

[('suspenseful', 0.8708139657974243),
 ('gripping', 0.85813307762146),
 ('exciting', 0.8081084489822388)]

**Huge**

In [551]:
model.most_similar(positive = ['huge'], topn = 3)

[('big', 0.7841212749481201),
 ('massive', 0.759773850440979),
 ('large', 0.6307077407836914)]

**Recommendation**

In [581]:
model.most_similar(positive = ['recommendation'], topn = 3)

[('advice', 0.758362352848053),
 ('estimation', 0.7475383281707764),
 ('suggestion', 0.7331489324569702)]

**Kind**

In [558]:
model.most_similar(positive = ['kind'], topn = 3)

[('sort', 0.8997496366500854),
 ('type', 0.7444096207618713),
 ('semblance', 0.5400595664978027)]

**Gang**

In [646]:
model.most_similar(positive = ['gang'], topn = 3)

[('mob', 0.7476179003715515),
 ('clan', 0.7398967146873474),
 ('tribe', 0.7168675661087036)]

Примеры, где синонимами являются только ТОП-**2** похожих слова: 

**Film**

In [553]:
model.most_similar(positive = ['film'], topn = 3)

[('movie', 0.946456789970398),
 ('picture', 0.7363325357437134),
 ('documentary', 0.7319446802139282)]

**Channel**

In [554]:
model.most_similar(positive = ['channel'], topn = 3)

[('network', 0.8229336738586426),
 ('cable', 0.8031548857688904),
 ('hbo', 0.7376188635826111)]

**Place**

In [680]:
model.most_similar(positive = ['home'], topn = 3)

[('vacation', 0.6129195690155029),
 ('house', 0.6092106103897095),
 ('dinner', 0.6032546758651733)]

Как видно, и documentry, и bbc, и country является похожими по смыслу словами, но не синонимами, так как это скорее слова, которые включается в большое множество film, channel и place, т.е. являются их подмножествами, а не эквивалентны им.

**Lucky**

In [559]:
model.most_similar(positive = ['lucky'], topn = 3)

[('fortunate', 0.7112294435501099),
 ('grateful', 0.5837315320968628),
 ('eager', 0.5461816787719727)]

В данном примере синонимом является уже только первое слово, что говорит о том, что most_similar не всегда подходит для определения похожих слов с точки зрения их лексического значения (синонимов). Также это будет подтверждено следующими примерами. 

Случаи, когда самым схожим словом является - **антоним** (love - hate, night - morning):

**Love**

In [674]:
model.most_similar(positive = ['love'], topn = 1)

[('hate', 0.5882089138031006)]

**Night**

In [676]:
model.most_similar(positive = ['night'], topn = 1)

[('morning', 0.7548598051071167)]

Можно заметить, что большую часть наших удачных примеров составляют прилагательные. Это связано с тем, что именно для прилагательных модель выдает лучшие результаты, то есть точнее определяет синонимы. Скорее всего, это связано с тем, что при написании рецензий используется огромное количество всевозможных прилагательных, что дает модели возможность лучше обучиться. Благодаря этому, для данного текста most_similar действительно подходит для определения синонимов слов, точность результатов которого значительно повышается, если часть речи проверяемого слова - прилагательное.  

## 3. Looking for associations

Приведите 5-10 примеров использования .most_similar для определения ассоциаций (А к Б, как В к?). Корректно ли найдены ассоциации?

In [562]:
model.most_similar(positive = ['hot'], negative = ['weather'], topn = 3)

[('cute', 0.5197433233261108),
 ('sexy', 0.5062525272369385),
 ('adorable', 0.4691374897956848)]

In [563]:
model.most_similar(positive = ['america'], topn = 3)

[('japan', 0.7715083956718445),
 ('china', 0.7648563385009766),
 ('korea', 0.7630176544189453)]

In [38]:
model.most_similar(positive = ['car', 'accident'], topn = 3)

[('plane', 0.7602980136871338),
 ('helicopter', 0.7549397945404053),
 ('truck', 0.7372640371322632)]

In [39]:
model.most_similar(positive = ['mother', 'father'], topn = 3)

[('husband', 0.8845164179801941),
 ('daughter', 0.8816182017326355),
 ('son', 0.8680322170257568)]

In [591]:
model.most_similar(positive = ['work', 'job'], negative = ['education'], topn = 3)

[('performance', 0.5002059936523438),
 ('direction', 0.44860243797302246),
 ('screenplay', 0.421019583940506)]

In [41]:
model.most_similar(positive = ['war', 'death', 'soldiers'], topn = 3)

[('vietnam', 0.7079989910125732),
 ('nazi', 0.7017005681991577),
 ('iraq', 0.7010694146156311)]

In [42]:
model.most_similar(positive = ['crime', 'blood', 'victim'], topn = 3)

[('maniac', 0.6700836420059204),
 ('rage', 0.6627383232116699),
 ('murder', 0.6604952216148376)]

In [590]:
model.most_similar(positive = ['film', 'comedy'], topn = 3)

[('movie', 0.8293472528457642),
 ('thriller', 0.7363054752349854),
 ('drama', 0.7097047567367554)]

In [589]:
model.most_similar(positive = ['cold'], topn = 3)

[('blooded', 0.6088370680809021),
 ('malevolent', 0.5584050416946411),
 ('ferocious', 0.5558328628540039)]

In [595]:
model.most_similar(positive = ['money', 'time'], topn = 3)

[('effort', 0.5436347126960754),
 ('resources', 0.5259729623794556),
 ('ticket', 0.5201836228370667)]

In [610]:
model.most_similar(positive = ['business', 'money', 'love'], topn = 3)

[('trouble', 0.5850033760070801),
 ('practice', 0.5755712985992432),
 ('respect', 0.5620672702789307)]

In [658]:
model.most_similar(positive = ['student', 'alcohol'], topn = 3)

[('heroin', 0.6636365652084351),
 ('dealer', 0.6579728126525879),
 ('addict', 0.6520422697067261)]

In [671]:
model.most_similar(positive = ['summer', 'friends', 'chill'], topn = 3)

[('meal', 0.6723304986953735),
 ('party', 0.6305789947509766),
 ('weekend', 0.6299766302108765)]

In [673]:
model.most_similar(positive = ['music','film'], topn = 3)

[('movie', 0.7839481830596924),
 ('soundtrack', 0.7768093347549438),
 ('animation', 0.7128516435623169)]

In [683]:
model.most_similar(positive = ['home'], topn = 3)

[('vacation', 0.6129195690155029),
 ('house', 0.6092106103897095),
 ('dinner', 0.6032546758651733)]

В целом, можно отметить, что модель хорошо обучилась, и может определять сложные ассоциации, делая это с высокой степенью качества. 

## 4. Looking for inappropriate words

Приведите 5-10 примеров использования .doesnt_match для определения лишнего слова. Корректно ли найдены лишние слова?

Model finds words that out of context

In [693]:
model.doesnt_match("university student sleep homework study teacher".split())

'sleep'

In [44]:
model.doesnt_match("pizza food popcorn salad fries character".split())

'character'

In [45]:
model.doesnt_match("movie film cinema hero actor singer".split())

'singer'

In [708]:
model.doesnt_match("jews genocide fun pain prison kill".split())

'fun'

In [719]:
model.doesnt_match("tree plant grass leaf green animal".split())

'animal'

In [47]:
model.doesnt_match("love joy like enjoy excite boring ".split())

'boring'

In [704]:
model.doesnt_match("throw grow blow follow borrow wow".split())

'wow'

Обученная модель умеет даже отличать мужские имена от женских:

In [716]:
model.doesnt_match("mary joan caroline john malisa keith".split())

'john'

Как и в предыдущих случаях, модель отработала на ура :) В результате использования метода .doesnt_match удается не только выделять слова, не подходящие по контексту, но и выделять более сложные зависимости. Такие, как, например, дифференциация женских имен от мужских или исключение слова по части речи.

## 5. Checking commutativity rule

Попробуйте найти такие пары и тройки слов, для которых:
* не выполняются свойства коммутативности и транзитивности относительно операции определения близких слов.
* выполняются свойства коммутативности и транзитивности отно-сительно операции определения близких слов.

Обозначим отношение “входить в топ-3 по.most_similar” символом ○.<br>
**Коммутативность:** x ○ y => y ○ x <br>
**Транзитивность:** x ○ y, y ○ z => x ○ z

Пары слов, для которых не выполняется свойство коммутативности

**Wild** and **Swamp**

In [758]:
model.most_similar(positive = ['wild'], topn = 3)

[('prehistoric', 0.5785297155380249),
 ('swamp', 0.5692760944366455),
 ('chic', 0.5658150315284729)]

In [760]:
model.most_similar(positive = ['swamp'], topn = 3)

[('jungle', 0.8163518309593201),
 ('cave', 0.7901890277862549),
 ('sewer', 0.7900116443634033)]

В топ-3 близких слов для Swamp не входит Wild, но входят слова, которые являются ассоциациями к Swamp в значении "болото".

**Space** and **Abyss**

In [741]:
model.most_similar(positive = ['space'], topn = 3)

[('outer', 0.6296460032463074),
 ('shuttle', 0.6139154434204102),
 ('abyss', 0.5848075151443481)]

In [742]:
model.most_similar(positive = ['abyss'], topn = 3)

[('excursion', 0.7219734191894531),
 ('vault', 0.7157360315322876),
 ('overdrive', 0.7106931209564209)]

**Ginger** and **Carrie**

In [744]:
model.most_similar(positive = ['ginger'], topn = 3)

[('rogers', 0.7238156199455261),
 ('astaire', 0.717879056930542),
 ('carrie', 0.685380756855011)]

In [746]:
model.most_similar(positive = ['carrie'], topn = 3)

[('sommer', 0.8206186890602112),
 ('claudia', 0.8128747344017029),
 ('martha', 0.8109422922134399)]

Как видно, Carrie входит в ТОП-3 похожих слов для Ginger, а вот Ginger не в ходит в ТОП-3 Carrie, зато в него входят другие имена, из чего делаем вывод, о том какие имена ближе друг к другу по смыслу, а также то, что Ginger в контексте наших данных встречалось чаще как имя или фамилия, нежели как "имбирь".

Еще один пример слов, которые не обладают свойством коммутативности: 

**South** and **Africa**

In [749]:
model.most_similar(positive = ['south'], topn = 3)

[('north', 0.7932907342910767),
 ('africa', 0.7536367177963257),
 ('korea', 0.736842930316925)]

In [756]:
model.most_similar(positive = ['africa'], topn = 3)

[('europe', 0.8369371891021729),
 ('korea', 0.8186471462249756),
 ('france', 0.8143318295478821)]

Для слова South близким является Africa, но для Africa похожими являются уже названия частей света, стран.

Пара слов, для которых выполняется свойство коммутативности

**Lovely** and **Gorgeous**

In [51]:
model.most_similar(positive = ['lovely'], topn = 3)

[('gorgeous', 0.8244462013244629),
 ('beautiful', 0.7893744707107544),
 ('sexy', 0.6904053688049316)]

In [52]:
model.most_similar(positive = ['gorgeous'], topn = 3)

[('beautiful', 0.843631386756897),
 ('lovely', 0.8244462013244629),
 ('stunning', 0.8003969192504883)]

**Superb** and **Fabulous**

In [783]:
model.most_similar(positive = ['superb'], topn = 3)

[('terrific', 0.8776928782463074),
 ('fabulous', 0.8636777400970459),
 ('fantastic', 0.860214352607727)]

In [786]:
model.most_similar(positive = ['fabulous'], topn = 3)

[('superb', 0.8636777400970459),
 ('fantastic', 0.8555989265441895),
 ('terrific', 0.8529516458511353)]

Как уже было замечено ранее, при написании рецензий авторы чаще всего используют прилагательные, поэтому эта часть речи, а особенно наиболее популярные слова в больших случаях обладает свойством коммутативности, что подтверждается нашим примером с lovely и gorgeous.

Проверим транзитивность: случай, когда свойство выполняется:

In [53]:
model.most_similar(positive = ['film'], topn = 3)

[('movie', 0.946456789970398),
 ('picture', 0.7363325357437134),
 ('documentary', 0.7319446802139282)]

In [54]:
model.most_similar(positive = ['movie'], topn = 3)

[('film', 0.946456789970398),
 ('flick', 0.7504115700721741),
 ('picture', 0.656548023223877)]

In [55]:
model.most_similar(positive = ['flick'], topn = 3)

[('movie', 0.7504115700721741),
 ('thriller', 0.7113186717033386),
 ('film', 0.7006672620773315)]

В данном случае выполняются свойства коммутативности и транзитивности film, movie коммутативны, а film и flick транзитивны

И, наконец, случай, когда свойство транзитивности не выполняется:

In [56]:
model.most_similar(positive = ['cat'], topn = 3)

[('sheep', 0.767533540725708),
 ('dog', 0.7201465368270874),
 ('tiger', 0.70553058385849)]

In [57]:
model.most_similar(positive = ['dog'], topn = 3)

[('cat', 0.720146656036377),
 ('demon', 0.6932040452957153),
 ('horse', 0.6806365847587585)]

In [58]:
model.most_similar(positive = ['demon'], topn = 3)

[('robot', 0.8184897899627686),
 ('maniac', 0.7666633725166321),
 ('rat', 0.7578243017196655)]

В этом же случае коммутативность между cat и dog присутствует,но вот транзитивность между cat и demon уже не наблюдается.